In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('test2.modify2.csv')
df.columns

Index(['unit', 'Ps30', 'Nc', 'T50', 'phi', 'NRc', 'W32', 'P30', 'BPR', 'T30',
       'T24', 'W31', 'htBleed', 'NRf', 'Nf', 'RUL'],
      dtype='object')

In [3]:
features = ['Ps30', 'Nc', 'T50', 'phi', 'NRc', 'W32', 'P30', 'BPR', 'T30',
       'T24', 'W31', 'htBleed', 'NRf', 'Nf']
label = 'RUL'

In [4]:
train, test = df[0::2], df[1::2]
train, test = train.reset_index(), test.reset_index()
X_train, y_train = train[features], train[label]
X_test, y_test = test[features], test[label]

#### 모델 기획 및 수립
- XGBoost(속도를 개선함), CatBoost, LightGBM -> 트리 기반의 알고리즘 
- XGBoost : 내부적을 병렬 계산을 극대화함 -> 속도가 개선함 regularizer가 내부적으로 작동함 이후의 cat, light에 대비해 reference가 많다. 

In [5]:
from sklearn.ensemble import RandomForestRegressor as rf
from xgboost import XGBRegressor as xg
from lightgbm import LGBMRegressor as lg
from catboost import CatBoostRegressor as cb

In [6]:
m1, m2, m3, m4 = rf(), xg(), lg(), cb(silent= True)

In [7]:
m1.fit(X_train, y_train)

RandomForestRegressor()

In [8]:
m2.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [9]:
m3.fit(X_train, y_train)

LGBMRegressor()

In [10]:
m4.fit(X_train, y_train)

In [11]:
print(m1.score(X_train, y_train), m2.score(X_train, y_train), m3.score(X_train, y_train), m4.score(X_train, y_train))
print(m1.score(X_test, y_test), m2.score(X_test, y_test), m3.score(X_test, y_test), m4.score(X_test, y_test))

# 학습한것에 비해 성능이 너무 좋지않다.
# rul이 0에 가까울수록 기울기의 변동폭이 엄청 커진다. 
# 수명이 죽을 때가 오면 기울기가 훅훅 떨어진다. 
# 7일차의 그래프를 참고하자. 
# 로그함수나 루트를 사용하자...

0.9487398253143053 0.8906117568834724 0.7678763179899359 0.806670475771444
0.6283134728294472 0.589376213162057 0.6274747881162159 0.6287516693582487


In [12]:
# 로그 오차....

In [13]:
df['RUL_Log'] = df['RUL'].apply(lambda x : np.log(x))
df.head()

,unit,Ps30,Nc,T50,phi,NRc,W32,P30,BPR,T30,T24,W31,htBleed,NRf,Nf,RUL,RUL_Log
0,1,47.47,9046.19,1400.60,521.66,8138.62,23.4190,554.36,8.4195,1589.70,641.82,39.06,392,2388.02,2388.06,192,5.257495
1,1,47.49,9044.07,1403.14,522.28,8131.49,23.4236,553.75,8.4318,1591.82,642.15,39.00,392,2388.07,2388.04,191,5.252273
2,1,47.27,9052.94,1404.20,522.42,8133.23,23.3442,554.26,8.4178,1587.99,642.35,38.95,390,2388.03,2388.08,190,5.247024
3,1,47.13,9049.48,1401.87,522.86,8133.83,23.3739,554.45,8.3682,1582.79,642.35,38.88,392,2388.08,2388.11,189,5.241747
4,1,47.28,9055.15,1406.22,522.19,8133.80,23.4044,554.00,8.4294,1582.85,642.37,38.90,393,2388.04,2388.06,188,5.236442


In [14]:
features = ['Ps30', 'Nc', 'T50', 'phi', 'NRc', 'W32', 'P30', 'BPR', 'T30',
       'T24', 'W31', 'htBleed', 'NRf', 'Nf']
label = 'RUL_Log'

In [15]:
train, test = df[0::2], df[1::2]
train, test = train.reset_index(), test.reset_index()
X_train, y_train = train[features], train[label]
X_test, y_test = test[features], test[label]

In [16]:
m1, m2, m3, m4 = rf(), xg(), lg(), cb(silent= True)

In [17]:
m1.fit(X_train, y_train)

RandomForestRegressor()

In [18]:
m2.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [19]:
m3.fit(X_train, y_train)

LGBMRegressor()

In [20]:
m4.fit(X_train, y_train)

In [21]:
print(m1.score(X_train, y_train), m2.score(X_train, y_train), m3.score(X_train, y_train), m4.score(X_train, y_train))
print(m1.score(X_test, y_test), m2.score(X_test, y_test), m3.score(X_test, y_test), m4.score(X_test, y_test))

0.9793777144343612 0.9505269071781702 0.9059628093954009 0.9190637667017059
0.8504263562007587 0.8349611612604918 0.8535134200337309 0.8553732607091954


#### Hyperparameter tuning

In [22]:
sample_model = xg(reg_alpha = 0.9)
sample_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [23]:
sample_model.score(X_test, y_test)

0.8364505083228531

### Optimizer

In [24]:
def alpha_search(alpha):
    sample_model = xg(reg_alpha = alpha)
    sample_model.fit(X_train, y_train)
    return sample_model.score(X_test, y_test)

In [25]:
alpha_search(0.2)

0.8368009499247369

In [26]:
def hyper_param(ra, rl, md, lr):
    sample_model = xg(reg_alpha = ra, reg_lambda = rl, max_depth = int(md), learning_rate = lr)
    sample_model.fit(X_train, y_train)
    return sample_model.score(X_test, y_test)

In [27]:
hyper_param(0.2, 0.3, 10, 0.6)

0.7825444137486678

In [28]:
from bayes_opt import BayesianOptimization as bo

In [29]:
epsilon = 0.1 ** 10
float_range = (epsilon, 1 - epsilon)
pbound = { 'ra' : float_range, 'rl' : float_range, 'lr' : float_range, 'md' : (5, 20)}

optimizer = bo(f = hyper_param, pbounds = pbound)

In [30]:
optimizer.maximize(n_iter = 25)

|   iter    |  target   |    lr     |    md     |    ra     |    rl     |
-------------------------------------------------------------------------
| 1         | 0.7915    | 0.0294    | 17.66     | 0.2366    | 0.3002    |
| 2         | 0.8451    | 0.07952   | 15.22     | 0.9534    | 0.6362    |
| 3         | 0.7724    | 0.6757    | 11.67     | 0.4622    | 0.06811   |
| 4         | 0.7958    | 0.5901    | 12.78     | 0.6232    | 0.8068    |
| 5         | 0.729     | 0.9732    | 6.262     | 0.02305   | 0.5878    |
| 6         | 0.6832    | 1.0       | 15.12     | 1e-10     | 1e-10     |
| 7         | 0.7893    | 0.6072    | 13.51     | 0.7319    | 0.9331    |
| 8         | -15.6     | 1e-10     | 15.08     | 1.0       | 1.0       |
| 9         | 0.8365    | 0.1031    | 19.47     | 0.3754    | 0.362     |
| 10        | 0.7753    | 0.8075    | 6.149     | 0.5966    | 0.3316    |
| 11        | 0.8162    | 0.5815    | 5.299     | 0.6704    | 0.5475    |
| 12        | 0.8441    | 0.1002    | 

In [31]:
param_set = optimizer.max['params']

In [32]:
tuned_model = xg(reg_alpha = param_set['ra'],
  reg_lambda = param_set['rl'],
  learning_rate = param_set['lr'],
  max_depth = int(param_set['md']))
                 
tuned_model.fit(X_train, y_train)
tuned_model.score(X_test, y_test)

0.8467114142598033

In [33]:
print(tuned_model.score(X_train, y_train))

0.9470074347796928


In [34]:
# LightGBM

In [35]:
def hyper_param2(ra, rl, md, lr):
    sample_model = lg(reg_alpha = ra, reg_lambda = rl, max_depth = int(md), learning_rate = lr)
    sample_model.fit(X_train, y_train)
    return sample_model.score(X_test, y_test)

In [36]:
epsilon = 0.1 ** 10
float_range = (epsilon, 1 - epsilon)
pbound = { 'ra' : float_range, 'rl' : float_range, 'lr' : float_range, 'md' : (5, 20)}

optimizer = bo(f = hyper_param2, pbounds = pbound)

In [37]:
optimizer.maximize(n_iter = 25)

|   iter    |  target   |    lr     |    md     |    ra     |    rl     |
-------------------------------------------------------------------------
| 1         | 0.8319    | 0.3723    | 19.43     | 0.8221    | 0.4562    |
| 2         | 0.8293    | 0.4107    | 7.375     | 0.9074    | 0.4004    |
| 3         | 0.8038    | 0.6264    | 13.79     | 0.07845   | 0.3959    |
| 4         | 0.8283    | 0.3863    | 12.13     | 0.01184   | 0.528     |
| 5         | 0.8543    | 0.06522   | 10.03     | 0.8253    | 0.6914    |
| 6         | -2.899e-0 | 1e-10     | 9.841     | 1e-10     | 0.5321    |
| 7         | 0.8298    | 0.3906    | 15.96     | 0.2598    | 0.8223    |
| 8         | 0.7545    | 0.9978    | 9.139     | 0.8057    | 0.7673    |
| 9         | 0.7655    | 0.9172    | 15.35     | 0.6771    | 0.9105    |
| 10        | 0.8227    | 0.4576    | 12.89     | 0.5638    | 0.7559    |
| 11        | 0.8222    | 0.4732    | 11.05     | 0.2514    | 0.4467    |
| 12        | 0.8182    | 0.4837    | 

In [38]:
param_set2 = optimizer.max['params']

In [39]:
tuned_model2 = xg(reg_alpha = param_set['ra'],
  reg_lambda = param_set['rl'],
  learning_rate = param_set['lr'],
  max_depth = int(param_set['md']))
                 
tuned_model2.fit(X_train, y_train)
tuned_model2.score(X_test, y_test)

0.8467114142598033

In [40]:
print(tuned_model2.score(X_train, y_train))

0.9470074347796928


In [41]:
# CatBoost

In [42]:
def hyper_param(depth, llr):
    sample_model = cb(depth = int(depth), l2_leaf_reg = llr, silent = True)
    sample_model.fit(X_train, y_train)
    return sample_model.score(X_test, y_test)

In [43]:
pbounds = { 'depth' : (4, 10), 'llr' : float_range}
optimizer = bo(f = hyper_param, pbounds = pbounds)

In [44]:
optimizer.maximize()

|   iter    |  target   |   depth   |    llr    |
-------------------------------------------------
| 1         | 0.857     | 4.874     | 0.7666    |
| 2         | 0.8576    | 7.397     | 0.5681    |
| 3         | 0.857     | 8.738     | 0.754     |
| 4         | 0.8575    | 6.199     | 0.3464    |
| 5         | 0.8567    | 9.057     | 0.827     |
| 6         | 0.8568    | 6.855     | 0.4691    |
| 7         | 0.8577    | 7.567     | 0.6204    |
| 8         | 0.8568    | 4.255     | 0.8572    |
| 9         | 0.8574    | 7.72      | 0.3043    |
| 10        | 0.8577    | 7.569     | 1.0       |
| 11        | 0.8571    | 8.65      | 0.6474    |
| 12        | 0.8577    | 7.999     | 1.0       |
| 13        | 0.8575    | 5.681     | 0.5122    |
| 14        | 0.8571    | 5.696     | 1e-10     |
| 15        | 0.8567    | 9.096     | 0.8997    |
| 16        | 0.8577    | 7.801     | 0.7521    |
| 17        | 0.8575    | 5.973     | 1.0       |
| 18        | 0.8565    | 9.992     | 0.01954   |


In [45]:
optimizer.max['params']

{'depth': 7.776964552893718, 'llr': 0.9942512022177036}